In [ ]:
####
from sklearn.svm import SVR
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from scipy.stats.mstats import winsorize
from sklearn.cluster import KMeans
from pandas import to_datetime
#!pip install fbprophet
from fbprophet import Prophet
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
## Read in Dataset as df, get country code dummies, and change date_time

## Load Data Sets, Create Date & y
# Train
train = pd.read_csv('train_data.csv')
pd.to_datetime(train['ofd_date'], infer_datetime_format=True) 
train['y'] = train['Earlies_Exp'] - train['MNR_SNR_Exp']
# Test
test = pd.read_csv('test.csv')
pd.to_datetime(test['ofd_date'], infer_datetime_format=True) 
test['y'] = 0
# Only keep columns in trainingset that we have in testset
train = train[test.columns[1:]]

In [ ]:
## Remove Outliers Training Set
for column in ['y', 'OFD', 'Slam','Earlies_Rec','Rollover','Returns','R_Sideline','Sideline']:
  train[column] = winsorize(train[column], limits=(0.05, 0.05))

In [ ]:
## Add test to training data for processing (no processing happening that could create data leakage)
train = train.append(test)

In [ ]:
## Reset Index and Sort Data Set
train = train.sort_values(by = ['ofd_date', 'station_code'])
train = train.reset_index(drop = True)
train = train.drop(['Unnamed: 0'], axis = 1)
#train

In [ ]:
## Create Data Variables
# ID
train['Id'] =train["ofd_date"]+ "_" + train["station_code"].map(str)

# Month, Day, Weekday
train.ofd_date = pd.to_datetime(train.ofd_date)
month = train['ofd_date'].dt.month
day = train['ofd_date'].dt.day
weekday = train['ofd_date'].dt.weekday
train_dates = train.copy()
train_dates['day'] = day
train_dates['weekday'] = weekday
train_dates['month'] = month
train_dates.set_index('Id', inplace = True)
#train_dates

In [ ]:
## Drop Date and FC Codes
df = train_dates.drop(['ofd_date', 'fc_codes'], axis = 1)
#df

In [ ]:
## Create Lags for all the variables
# Define function
def create_lags(data, no_lags):
  for i in range(1, int(no_lags)):
    data["lag_{}_Earlies_Rec".format(i)] = data.Earlies_Rec.shift(i)
    data["lag_{}_OFD".format(i)] = data.OFD.shift(i)
    data["lag_{}_Slam".format(i)] = data.Slam.shift(i)
    data["lag_{}_Rollover".format(i)] = data.Rollover.shift(i)
    data["lag_{}_Returns".format(i)] = data.Returns.shift(i)
    data["lag_{}_Slideline".format(i)] = data.Sideline.shift(i)
  return data

# Create 21 Lags
df_lags = create_lags(df, 21)
#df_lags

In [ ]:
## Encoding Country Codes and making DC numerical
df_encoded = pd.get_dummies(df_lags, columns = ['country_code'])

df_encoded['DC'] = df_encoded['station_code'].str[1:]
df_encoded = df_encoded.drop(['station_code'], axis = 1)
df_encoded['DC'] = pd.to_numeric(df_encoded['DC'])
#df_encoded

In [ ]:
## Finding strong correlation
correlations = df_encoded.corr().sort_values('y')['y']
correlations = pd.DataFrame(correlations)
#correlations

In [ ]:
## Dropping Variables with certain correlations
corr = df_encoded.corr()
strong_var_pos = corr.loc[corr['y'] > 0 ].index.to_list()
strong_var_neg = corr.loc[corr['y'] < 0 ].index.to_list()
strong_var_pos.extend(strong_var_neg)
#strong_var_pos.append('DC')
#strong_var_pos.append('day')

In [ ]:
## Only select the strong correlated variables + Days & DC
df_strong =  df_encoded[[c for c in df_encoded.columns if c in strong_var_pos]]
#df_strong

In [ ]:
## Create the Three batches
df_train = df_strong.loc[df_strong['month'] != 7]
df_train = df_strong.loc[df_strong['month'] != 6]
df_train = df_train.loc[df_strong['month'] != 2]
df_test = df_strong.loc[df_strong['month'] == 6]

In [ ]:
## Create the X and Y for the threee batches
scaler = StandardScaler()
y_train = df_train['y']
y_test = df_test['y']
X_train = df_train.drop(['y'], axis = 1)
X_test = df_test.drop(['y'], axis = 1)

In [ ]:
## Run and Test the Model
regr =  make_pipeline(LinearRegression())
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [ ]:
#mean_squared_error(y_test, y_pred, squared=False)

432.8918894222631

In [ ]:
## Create Expected Column
df = pd.DataFrame(y_pred, columns = ['Expected'])
#df

In [ ]:
## Create Id Column
df_test = df_test[['y']]
df_test['Id'] = df_test.index
#df_test = df_test.reset_index(drop=True)

In [ ]:
# Create Final DF from Expected & ID
final=pd.concat([df_test,df],axis=1,ignore_index=True)
final = final[[1,2]]
final

In [ ]:
## Export Reults for Sub,ission
from google.colab import drive
drive.mount('drive/', force_remount=False)
final.to_csv('linear_regression.csv')
!cp linear_regression.csv "drive/My Drive/"

Mounted at drive/
